In [140]:
from importlib import reload
import utils
from utils import bandits, snowden
reload(utils)
reload(utils.bandits)
reload(utils.snowden)
from utils.bandits import exact_gittins

from templates import neural as neural_agent
reload(neural_agent)

neural_agent_class = neural_agent.NeuralAgent

In [96]:
datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/nagiss/20201226/', 1/10)

sessions/nagiss/20201226/ ['sessions/nagiss/20201226/8389416.json', 'sessions/nagiss/20201226/8528396.json', 'sessions/nagiss/20201226/8415103.json', 'sessions/nagiss/20201226/8354920.json', 'sessions/nagiss/20201226/8499335.json', 'sessions/nagiss/20201226/8441958.json', 'sessions/nagiss/20201226/8377070.json', 'sessions/nagiss/20201226/8513521.json', 'sessions/nagiss/20201226/8364900.json', 'sessions/nagiss/20201226/8428032.json', 'sessions/nagiss/20201226/8485987.json', 'sessions/nagiss/20201226/8455991.json', 'sessions/nagiss/20201226/8471153.json', 'sessions/nagiss/20201226/8401973.json']


In [98]:
datasets[0].X.shape, datasets[1].X.shape

((25987, 36), (1999, 36))

In [99]:
train = snowden.resample_eq(datasets[0])
val = snowden.resample_eq(datasets[1])

mean before resampling: 0.31915957978989495
mean after resampling: 0.5
mean before resampling: 0.3166583291645823
mean after resampling: 0.5


In [100]:
train.X.shape, val.X.shape

((16588, 36), (1266, 36))

In [17]:
import torch
from torch import nn
from torch import optim

import numpy as np

In [67]:
def learn(train, val, model_ff, epochs=5, batch_size=64, shuffle=True, freq=10,lr=1e-3):          
    np.random.seed(1)
    ids_nn = np.arange(train.X.shape[0])

#     criterion = nn.BCELoss()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model_ff.parameters(), lr=lr)

    time_for_print_loss = lambda i: (i + 1) % freq == 0
    

    for epoch in np.arange(epochs):
        np.random.shuffle(ids_nn)
        X_train_shuffled = train.X[ids_nn]
        y_train_shuffled = train.y[ids_nn]
        
        model_ff.train(True)

        for b in np.arange(0, X_train_shuffled.shape[0], batch_size):
            X_batch = torch.FloatTensor(X_train_shuffled[b:b+batch_size])
            y_batch = torch.FloatTensor(y_train_shuffled[b:b+batch_size])

            optimizer.zero_grad()
            y_pred_logits = model_ff(X_batch).reshape(-1)

            loss = criterion(y_pred_logits, y_batch)
            loss.backward()

            optimizer.step()

            if (b // batch_size + 1) % freq == 0:
                print('train loss in %d epoch in %d batch: %.5f' %
                  (epoch + 1, b // batch_size + 1, loss.item()))

        val_loss = 0
        its = 0
        model_ff.train(False)
        for b in np.arange(0, val.X.shape[0], batch_size):
            its += 1
            X_batch = torch.FloatTensor(val.X[b:b+batch_size])
            y_batch = torch.FloatTensor(val.y[b:b+batch_size])
            with torch.no_grad():
                y_pred_logits = model_ff(X_batch).reshape(-1)
            loss = criterion(y_pred_logits, y_batch)
            val_loss += loss.item()
        val_loss /= its
        print('val loss in %d epoch: %.5f' % (epoch + 1, val_loss))

In [19]:
class NNWithCusomFeatures(nn.Module):
    def __init__(self, INPUT_F, DROP_P, H):
        super().__init__()
        INPUT_F_C = INPUT_F + 2 * INPUT_F
        self.model_ff =  nn.Sequential(
            nn.BatchNorm1d(INPUT_F_C),
            nn.Dropout(DROP_P),
            nn.Linear(INPUT_F_C, H),
            nn.Sigmoid(),
            nn.Dropout(DROP_P),
            nn.Linear(H, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        lg = torch.log(1 + torch.abs(x))
        sn = torch.sin(x)
        input_x = torch.cat([x, lg, sn], axis=1)
        return self.model_ff(input_x)

In [155]:
INPUT_F = 36
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train, val, model, freq=40, batch_size=32,lr=1e-5, epochs=7)

train loss in 1 epoch in 40 batch: 0.695
train loss in 1 epoch in 80 batch: 0.680
train loss in 1 epoch in 120 batch: 0.696
train loss in 1 epoch in 160 batch: 0.661
train loss in 1 epoch in 200 batch: 0.702
train loss in 1 epoch in 240 batch: 0.665
train loss in 1 epoch in 280 batch: 0.645
train loss in 1 epoch in 320 batch: 0.680
train loss in 1 epoch in 360 batch: 0.683
train loss in 1 epoch in 400 batch: 0.670
train loss in 1 epoch in 440 batch: 0.689
train loss in 1 epoch in 480 batch: 0.630
val loss in 1 epoch: 0.657
train loss in 2 epoch in 40 batch: 0.648
train loss in 2 epoch in 80 batch: 0.672
train loss in 2 epoch in 120 batch: 0.593
train loss in 2 epoch in 160 batch: 0.685
train loss in 2 epoch in 200 batch: 0.696
train loss in 2 epoch in 240 batch: 0.631
train loss in 2 epoch in 280 batch: 0.621
train loss in 2 epoch in 320 batch: 0.661
train loss in 2 epoch in 360 batch: 0.696
train loss in 2 epoch in 400 batch: 0.569
train loss in 2 epoch in 440 batch: 0.673
train loss 

In [156]:
torch.save(model.state_dict(), 'models/nagiss_v1')

In [174]:
# mse loss
INPUT_F = 36
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train, val, model, freq=40, batch_size=32,lr=1e-5, epochs=5)

train loss in 1 epoch in 40 batch: 0.246
train loss in 1 epoch in 80 batch: 0.244
train loss in 1 epoch in 120 batch: 0.248
train loss in 1 epoch in 160 batch: 0.233
train loss in 1 epoch in 200 batch: 0.242
train loss in 1 epoch in 240 batch: 0.238
train loss in 1 epoch in 280 batch: 0.239
train loss in 1 epoch in 320 batch: 0.235
train loss in 1 epoch in 360 batch: 0.245
train loss in 1 epoch in 400 batch: 0.240
train loss in 1 epoch in 440 batch: 0.248
train loss in 1 epoch in 480 batch: 0.216
val loss in 1 epoch: 0.234
train loss in 2 epoch in 40 batch: 0.234
train loss in 2 epoch in 80 batch: 0.233
train loss in 2 epoch in 120 batch: 0.199
train loss in 2 epoch in 160 batch: 0.247
train loss in 2 epoch in 200 batch: 0.256
train loss in 2 epoch in 240 batch: 0.219
train loss in 2 epoch in 280 batch: 0.221
train loss in 2 epoch in 320 batch: 0.227
train loss in 2 epoch in 360 batch: 0.239
train loss in 2 epoch in 400 batch: 0.194
train loss in 2 epoch in 440 batch: 0.240
train loss 

In [175]:
torch.save(model.state_dict(), 'models/nagiss_v2')

In [178]:
# v1
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

632.0 611.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
676.0 642.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
599.0 543.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
624.0 621.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
606.0 639.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
607.0 591.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
609.0 552.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
626.0 606.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
630.0 603.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
615.0 615.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py


(0.012154502623372141,
 20.1,
 25.34738645304482,
 0.9,
 'tmp/b_0.9817500612110714.py',
 'tmp/b_0.6737446183327064.py',
 array([632., 676., 599., 624., 606., 607., 609., 626., 630., 615.]),
 array([611., 642., 543., 621., 639., 591., 552., 606., 603., 615.]))

In [180]:
# v2
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text= utils.bandits.neural),
    verbose=True)

678.0 661.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
700.0 695.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
664.0 660.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
597.0 615.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
659.0 696.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
601.0 581.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
626.0 623.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
536.0 607.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
580.0 550.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
636.0 562.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py


(0.8180618145466889,
 2.7,
 37.11616898334202,
 0.7,
 'tmp/b_0.054213267519393926.py',
 'tmp/b_0.236142134925876.py',
 array([678., 700., 664., 597., 659., 601., 626., 536., 580., 636.]),
 array([661., 695., 660., 615., 696., 581., 623., 607., 550., 562.]))

In [6]:
# v2
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text= utils.bandits.neural),
    verbose=True)

657.0 685.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
717.0 717.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
691.0 721.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
629.0 671.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
624.0 662.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
677.0 627.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
579.0 572.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
594.0 621.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
617.0 593.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
629.0 634.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py


(0.31907279631279994,
 -8.9,
 28.246946737656444,
 0.4,
 'tmp/b_0.4681855452475341.py',
 'tmp/b_0.815911715993058.py',
 array([657., 717., 691., 629., 624., 677., 579., 594., 617., 629.]),
 array([685., 717., 721., 671., 662., 627., 572., 621., 593., 634.]))

### all nagiss games

In [7]:
datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/nagiss/20201226_v2/', 1/10)

sessions/nagiss/20201226_v2/ ['sessions/nagiss/20201226_v2/7994638.json', 'sessions/nagiss/20201226_v2/7991093.json', 'sessions/nagiss/20201226_v2/7988820.json', 'sessions/nagiss/20201226_v2/8386716.json', 'sessions/nagiss/20201226_v2/8038792.json', 'sessions/nagiss/20201226_v2/8233786.json', 'sessions/nagiss/20201226_v2/8073493.json', 'sessions/nagiss/20201226_v2/8012231.json', 'sessions/nagiss/20201226_v2/7995268.json', 'sessions/nagiss/20201226_v2/8046220.json', 'sessions/nagiss/20201226_v2/8321309.json', 'sessions/nagiss/20201226_v2/8102725.json', 'sessions/nagiss/20201226_v2/8146617.json', 'sessions/nagiss/20201226_v2/7986639.json', 'sessions/nagiss/20201226_v2/8029737.json', 'sessions/nagiss/20201226_v2/8089900.json', 'sessions/nagiss/20201226_v2/8354985.json', 'sessions/nagiss/20201226_v2/7995644.json', 'sessions/nagiss/20201226_v2/7998867.json', 'sessions/nagiss/20201226_v2/8257767.json', 'sessions/nagiss/20201226_v2/7992085.json', 'sessions/nagiss/20201226_v2/7987842.json', 's

In [11]:
datasets[0].X.shape, datasets[1].X.shape

((133933, 36), (13993, 36))

In [12]:
train = snowden.resample_eq(datasets[0])
val = snowden.resample_eq(datasets[1])

mean before resampling: 0.31643433657127074
mean after resampling: 0.5
mean before resampling: 0.3148002572714929
mean after resampling: 0.5


In [13]:
train.X.shape, val.X.shape

((84762, 36), (8810, 36))

In [ ]:
# class NNWithCusomFeatures(nn.Module):
#     def __init__(self, INPUT_F, DROP_P, H):
#         super().__init__()
#         INPUT_F_C = INPUT_F + 2 * INPUT_F
#         self.model_ff_1 =  nn.Sequential(
#             nn.BatchNorm1d(INPUT_F_C),
#             nn.Dropout(DROP_P),
#             nn.Linear(INPUT_F_C, H),
#             nn.Sigmoid(),
#             nn.Dropout(DROP_P)
#         )
#         self.model_ff_2 =  nn.Sequential(
#             nn.Linear(H, 1),
#             nn.Sigmoid()
#         )

#     def forward(self, x):
#         lg = torch.log(1 + torch.abs(x))
#         sn = torch.sin(x)
#         input_x = torch.cat([x, lg, sn], axis=1)
#         out1 =  self.model_ff_1(input_x)

In [76]:
# mse loss
INPUT_F = 36
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train, val, model, freq=500, batch_size=32,lr=1e-5, epochs=5)

train loss in 1 epoch in 500 batch: 0.24326
train loss in 1 epoch in 1000 batch: 0.23294
train loss in 1 epoch in 1500 batch: 0.25006
train loss in 1 epoch in 2000 batch: 0.20686
train loss in 1 epoch in 2500 batch: 0.21678
val loss in 1 epoch: 0.22331
train loss in 2 epoch in 500 batch: 0.26177
train loss in 2 epoch in 1000 batch: 0.19144
train loss in 2 epoch in 1500 batch: 0.17022
train loss in 2 epoch in 2000 batch: 0.23671
train loss in 2 epoch in 2500 batch: 0.22735
val loss in 2 epoch: 0.22240
train loss in 3 epoch in 500 batch: 0.18364
train loss in 3 epoch in 1000 batch: 0.25060
train loss in 3 epoch in 1500 batch: 0.25504
train loss in 3 epoch in 2000 batch: 0.21704
train loss in 3 epoch in 2500 batch: 0.20117
val loss in 3 epoch: 0.22189
train loss in 4 epoch in 500 batch: 0.17385
train loss in 4 epoch in 1000 batch: 0.22882
train loss in 4 epoch in 1500 batch: 0.22496
train loss in 4 epoch in 2000 batch: 0.20567
train loss in 4 epoch in 2500 batch: 0.22413
val loss in 4 epo

In [77]:
torch.save(model.state_dict(), 'models/nagiss_v3')

In [79]:
# v3
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

619.0 538.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
552.0 574.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
641.0 652.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
631.0 601.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
703.0 651.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
643.0 666.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
617.0 628.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
607.0 574.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
638.0 660.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
618.0 632.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py


(0.4021809806045401,
 9.3,
 35.105697543276364,
 0.4,
 'tmp/b_0.5394941534468733.py',
 'tmp/b_0.9677390896108925.py',
 array([619., 552., 641., 631., 703., 643., 617., 607., 638., 618.]),
 array([538., 574., 652., 601., 651., 666., 628., 574., 660., 632.]))

In [116]:
#with mean features

datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/nagiss/20201226_v2/', 1/10)

sessions/nagiss/20201226_v2/ ['sessions/nagiss/20201226_v2/7994638.json', 'sessions/nagiss/20201226_v2/7991093.json', 'sessions/nagiss/20201226_v2/7988820.json', 'sessions/nagiss/20201226_v2/8386716.json', 'sessions/nagiss/20201226_v2/8038792.json', 'sessions/nagiss/20201226_v2/8233786.json', 'sessions/nagiss/20201226_v2/8073493.json', 'sessions/nagiss/20201226_v2/8012231.json', 'sessions/nagiss/20201226_v2/7995268.json', 'sessions/nagiss/20201226_v2/8046220.json', 'sessions/nagiss/20201226_v2/8321309.json', 'sessions/nagiss/20201226_v2/8102725.json', 'sessions/nagiss/20201226_v2/8146617.json', 'sessions/nagiss/20201226_v2/7986639.json', 'sessions/nagiss/20201226_v2/8029737.json', 'sessions/nagiss/20201226_v2/8089900.json', 'sessions/nagiss/20201226_v2/8354985.json', 'sessions/nagiss/20201226_v2/7995644.json', 'sessions/nagiss/20201226_v2/7998867.json', 'sessions/nagiss/20201226_v2/8257767.json', 'sessions/nagiss/20201226_v2/7992085.json', 'sessions/nagiss/20201226_v2/7987842.json', 's

  0%|          | 0/67 [00:00<?, ?it/s]

error while parse session sessions/nagiss/20201226_v2/.ipynb_checkpoints/8002300-checkpoint.json : 


100%|██████████| 7/7 [01:19<00:00, 11.38s/it]


In [117]:
datasets[0].X.shape, datasets[1].X.shape

((133933, 72), (13993, 72))

In [118]:
train = snowden.resample_eq(datasets[0])
val = snowden.resample_eq(datasets[1])

mean before resampling: 0.3157548923715589
mean after resampling: 0.5
mean before resampling: 0.3213035088973058
mean after resampling: 0.5


In [122]:
# mse loss
INPUT_F = 72
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train, val, model, freq=500, batch_size=32,lr=1e-5, epochs=2)

train loss in 1 epoch in 500 batch: 0.21463
train loss in 1 epoch in 1000 batch: 0.21573
train loss in 1 epoch in 1500 batch: 0.21318
train loss in 1 epoch in 2000 batch: 0.23660
train loss in 1 epoch in 2500 batch: 0.24215
val loss in 1 epoch: 0.22201
train loss in 2 epoch in 500 batch: 0.23591
train loss in 2 epoch in 1000 batch: 0.20482
train loss in 2 epoch in 1500 batch: 0.23344
train loss in 2 epoch in 2000 batch: 0.16600
train loss in 2 epoch in 2500 batch: 0.21033
val loss in 2 epoch: 0.22148


In [123]:
torch.save(model.state_dict(), 'models/nagiss_v4')

In [124]:
# v4
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

667.0 754.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
592.0 654.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
594.0 554.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
635.0 670.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
568.0 609.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
575.0 603.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
603.0 606.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
668.0 660.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
613.0 595.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py
598.0 582.0 tmp/b_0.6401838703031111.py tmp/b_0.7108320478829738.py


(0.14605214374251638,
 -17.4,
 37.852873074576515,
 0.4,
 'tmp/b_0.6401838703031111.py',
 'tmp/b_0.7108320478829738.py',
 array([667., 592., 594., 635., 568., 575., 603., 668., 613., 598.]),
 array([754., 654., 554., 670., 609., 603., 606., 660., 595., 582.]))

In [127]:
# # v4 vs default + mean features
# utils.bandits.compare(
#     utils.bandits.Agent(text=neural_with_nagiss),
#     utils.bandits.Agent(text=utils.bandits.neural),
#     verbose=True)

In [139]:
# v3 as feature
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

637.0 624.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
652.0 693.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
600.0 628.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
597.0 611.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
646.0 634.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
624.0 627.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
595.0 599.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
605.0 596.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
664.0 672.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
628.0 608.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py


(0.44715964885505466,
 -4.4,
 18.30409790183608,
 0.4,
 'tmp/b_0.00783945348032078.py',
 'tmp/b_0.08316848165655322.py',
 array([637., 652., 600., 597., 646., 624., 595., 605., 664., 628.]),
 array([624., 693., 628., 611., 634., 627., 599., 596., 672., 608.]))

In [141]:
# v2 as feature
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

617.0 640.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
644.0 619.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
679.0 694.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
645.0 617.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
601.0 629.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
628.0 617.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
639.0 634.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
651.0 658.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
736.0 647.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
586.0 588.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py


(0.41522101422787383,
 8.3,
 32.21505859066533,
 0.5,
 'tmp/b_0.2329175121645335.py',
 'tmp/b_0.9947346263005474.py',
 array([617., 644., 679., 645., 601., 628., 639., 651., 736., 586.]),
 array([640., 619., 694., 617., 629., 617., 634., 658., 647., 588.]))

In [137]:
# v4 as feature
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss,use_mean=True,True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

627.0 664.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
563.0 601.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
579.0 583.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
673.0 630.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
648.0 693.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
633.0 632.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
625.0 616.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
607.0 577.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
625.0 655.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
617.0 651.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py


(0.25859498040269646,
 -10.5,
 29.391325250828686,
 0.4,
 'tmp/b_0.6232887914692327.py',
 'tmp/b_0.7812314428145001.py',
 array([627., 563., 579., 673., 648., 633., 625., 607., 625., 617.]),
 array([664., 601., 583., 630., 693., 632., 616., 577., 655., 651.]))

In [142]:
# v2 as feature vs only v2
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss=True", "")
neural_with_only_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_with_only_nagiss),
    verbose=True)

538.0 550.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
633.0 611.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
589.0 594.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
614.0 638.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
631.0 664.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
656.0 644.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
529.0 583.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
610.0 635.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
641.0 625.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
664.0 632.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py


(0.38850022498604,
 -7.1,
 26.036320784627,
 0.4,
 'tmp/b_0.03981686451690292.py',
 'tmp/b_0.8435219654502392.py',
 array([538., 633., 589., 614., 631., 656., 529., 610., 641., 664.]),
 array([550., 611., 594., 638., 664., 644., 583., 635., 625., 632.]))